In [458]:
import numpy as np
import pandas as pd

from fbprophet import Prophet

import keras
from keras import models
from keras import layers

from keras.layers import Dense, Activation, GRU, Dropout, Flatten, Input
from keras.models import Sequential, Model

In [6]:
macro = pd.read_csv('/home/ubuntu/csvs/macro_pivot.csv', parse_dates=True, infer_datetime_format=True)

In [8]:
macro = macro.set_index('date')

In [20]:
len(macro)

1825

In [42]:
macro.index = pd.to_datetime(macro.index)

In [44]:
macro.index[0]

Timestamp('2014-03-17 00:00:00')

In [3]:
etfs = pd.read_csv('/home/ubuntu/csvs/etfs_pivot.csv', parse_dates=True, infer_datetime_format=True)

In [9]:
etfs = etfs.set_index('date')

In [40]:
etfs.index = pd.to_datetime(etfs.index)

In [499]:
etfs

,AAXJ,ACIM,ACWI,ACWV,ACWX,ADRD,ADRE,AFK,AGG,AGQ,...,XTN,YANG,YCS,YINN,YMLI,YMLP,YYY,ZMLP,ZROZ,ZSL
date,,,,,,,,,,,,,,,,,,,,,
2014-03-17,51.8998,55.3618,51.1969,56.4986,39.9088,20.5917,31.1902,27.6914,95.1937,73.2500,...,41.4988,741.9276,65.7700,18.0676,14.6311,53.9279,15.0075,25.0558,79.4274,37.1900
2014-03-18,52.2619,55.6217,51.5374,56.8371,40.1901,20.7505,31.5912,27.7092,95.3084,70.9400,...,41.7226,704.7079,65.3100,18.8990,14.7005,53.8652,15.0523,25.2310,79.7748,38.2300
2014-03-19,51.3565,55.6217,51.0536,56.3470,39.6539,20.5163,31.3451,27.2374,94.8848,69.4300,...,41.4290,737.4106,66.7300,18.0382,14.5478,53.5998,14.9886,25.0308,79.1755,39.2064
2014-03-20,51.1120,55.6217,51.1700,56.2667,39.6012,20.4829,31.4864,27.5400,94.8142,67.5200,...,41.3328,745.2790,66.5800,17.9404,14.5339,53.6893,14.9440,25.1371,79.4534,40.1050
2014-03-21,51.4742,55.2919,51.0536,56.1151,39.5661,20.4949,31.5548,27.9050,94.9466,67.5200,...,41.3183,682.5782,66.2200,19.3881,14.5061,53.6893,14.9695,25.1034,80.3221,40.1450
2014-03-22,51.4742,55.2919,51.0536,56.1151,39.5661,20.4949,31.5548,27.9050,94.9466,67.5200,...,41.3183,682.5782,66.2200,19.3881,14.5061,53.6893,14.9695,25.1034,80.3221,40.1450
2014-03-23,51.4742,55.2919,51.0536,56.1151,39.5661,20.4949,31.5548,27.9050,94.9466,67.5200,...,41.3183,682.5782,66.2200,19.3881,14.5061,53.6893,14.9695,25.1034,80.3221,40.1450
2014-03-24,51.9269,55.1304,50.9908,56.2756,39.6715,20.4641,31.8920,27.7715,95.0878,65.3500,...,41.1997,647.6624,66.2500,20.4153,14.4367,53.2418,14.9122,25.0057,81.4687,41.4350
2014-03-25,52.3344,55.3084,51.3851,56.5789,40.1373,20.6669,32.1837,27.7804,94.9907,65.4400,...,41.3358,633.6470,66.3700,20.8065,14.4235,53.2717,14.9635,25.0121,81.0343,41.3350


In [22]:
date_ranges = []

date_ranges.append(pd.date_range(start='2014-03-17', end='2014-06-17').tolist())
date_ranges.append(pd.date_range(start='2014-06-18', end='2014-09-17').tolist())
date_ranges.append(pd.date_range(start='2014-09-18', end='2014-12-17').tolist())
date_ranges.append(pd.date_range(start='2014-12-18', end='2015-03-17').tolist())

date_ranges.append(pd.date_range(start='2015-03-18', end='2015-06-17').tolist())
date_ranges.append(pd.date_range(start='2015-06-18', end='2015-09-17').tolist())
date_ranges.append(pd.date_range(start='2015-09-18', end='2015-12-17').tolist())
date_ranges.append(pd.date_range(start='2015-12-18', end='2016-03-17').tolist())

date_ranges.append(pd.date_range(start='2016-03-18', end='2016-06-17').tolist())
date_ranges.append(pd.date_range(start='2016-06-18', end='2016-09-17').tolist())
date_ranges.append(pd.date_range(start='2016-09-18', end='2016-12-17').tolist())
date_ranges.append(pd.date_range(start='2016-12-18', end='2017-03-17').tolist())

date_ranges.append(pd.date_range(start='2017-03-18', end='2017-06-17').tolist())
date_ranges.append(pd.date_range(start='2017-06-18', end='2017-09-17').tolist())
date_ranges.append(pd.date_range(start='2017-09-18', end='2017-12-17').tolist())
date_ranges.append(pd.date_range(start='2017-12-18', end='2018-03-17').tolist())

date_ranges.append(pd.date_range(start='2018-03-18', end='2018-06-17').tolist())
date_ranges.append(pd.date_range(start='2018-06-18', end='2018-09-17').tolist())
date_ranges.append(pd.date_range(start='2018-09-18', end='2018-12-17').tolist())
date_ranges.append(pd.date_range(start='2018-12-18', end='2019-03-15').tolist())

In [535]:
spy = pd.DataFrame(etfs['SPY'])

In [534]:
ts_model = Prophet()

In [47]:
type(spy.index[0])

pandas._libs.tslib.Timestamp

In [536]:
df = spy[spy.index.isin(date_ranges[0])]

In [52]:
df

,SPY
date,
2014-03-17,168.0166
2014-03-18,169.2159
2014-03-19,168.3142
2014-03-20,169.2944
2014-03-21,168.6401
2014-03-22,168.6401
2014-03-23,168.6401
2014-03-24,167.9427
2014-03-25,168.7397


In [537]:
df = df.reset_index()

In [538]:
df.columns = ['ds', 'y']
ts_model.fit(df)

INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/home/ubuntu/anaconda3/envs/env_zipline/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [539]:
future = ts_model.make_future_dataframe(periods = len(date_ranges[1]))

In [540]:
forecast = ts_model.predict(future)

In [541]:
forecast

,ds,trend,trend_lower,trend_upper,yhat_lower,yhat_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yhat
0,2014-03-17,168.352846,168.352846,168.352846,167.207658,169.323714,-0.103966,-0.103966,-0.103966,0.0,0.0,0.0,-0.103966,-0.103966,-0.103966,168.248880
1,2014-03-18,168.376175,168.376175,168.376175,167.457944,169.631904,0.204145,0.204145,0.204145,0.0,0.0,0.0,0.204145,0.204145,0.204145,168.580320
2,2014-03-19,168.399504,168.399504,168.399504,167.771774,169.892887,0.430755,0.430755,0.430755,0.0,0.0,0.0,0.430755,0.430755,0.430755,168.830259
3,2014-03-20,168.422833,168.422833,168.422833,167.366744,169.537457,0.094329,0.094329,0.094329,0.0,0.0,0.0,0.094329,0.094329,0.094329,168.517162
4,2014-03-21,168.446162,168.446162,168.446162,167.241318,169.407219,-0.129239,-0.129239,-0.129239,0.0,0.0,0.0,-0.129239,-0.129239,-0.129239,168.316923
5,2014-03-22,168.469491,168.469491,168.469491,167.271961,169.317769,-0.190265,-0.190265,-0.190265,0.0,0.0,0.0,-0.190265,-0.190265,-0.190265,168.279226
6,2014-03-23,168.492820,168.492820,168.492820,167.049050,169.214916,-0.305759,-0.305759,-0.305759,0.0,0.0,0.0,-0.305759,-0.305759,-0.305759,168.187061
7,2014-03-24,168.516149,168.516149,168.516149,167.328670,169.419984,-0.103966,-0.103966,-0.103966,0.0,0.0,0.0,-0.103966,-0.103966,-0.103966,168.412184
8,2014-03-25,168.539478,168.539478,168.539478,167.691925,169.789326,0.204145,0.204145,0.204145,0.0,0.0,0.0,0.204145,0.204145,0.204145,168.743623
9,2014-03-26,168.562807,168.562807,168.562807,167.990749,170.107537,0.430755,0.430755,0.430755,0.0,0.0,0.0,0.430755,0.430755,0.430755,168.993562


In [61]:
resid = pd.DataFrame(index=date_ranges[1])

In [72]:
preds = np.array(forecast[forecast['ds'].isin(date_ranges[1])]['yhat'])
actuals = np.array(spy[spy.index.isin(date_ranges[1])]['SPY'])

In [74]:
actuals

array([177.7468, 177.0993, 178.3157, 178.3157, 178.3157, 178.2611,
       177.1872, 177.9881, 177.8606, 178.2065, 178.2065, 178.2065,
       178.1155, 179.3076, 179.4896, 180.3724, 180.3724, 180.3724,
       180.3724, 179.7445, 178.5887, 179.3895, 178.6797, 178.9254,
       178.9254, 178.9254, 179.8264, 179.4896, 180.154 , 178.1064,
       179.9264, 179.9264, 179.9264, 179.5897, 180.3724, 180.7728,
       180.7819, 179.9356, 179.9356, 179.9356, 180.0084, 179.2348,
       179.2621, 175.722 , 175.1851, 175.1851, 175.1851, 176.4501,
       174.7392, 174.7938, 173.8473, 175.8585, 175.8585, 175.8585,
       176.3636, 176.1224, 177.3146, 178.1519, 178.1155, 178.1155,
       178.1155, 179.6079, 180.5453, 181.0276, 181.5555, 181.2733,
       181.2733, 181.2733, 182.1925, 182.3108, 182.238 , 182.1379,
       182.6566, 182.6566, 182.6566, 182.6566, 182.5656, 182.4655,
       182.2017, 183.0206, 183.0206, 183.0206, 182.5474, 181.3916,
       182.0742, 182.2835, 181.2187, 181.2187, 181.2187, 181.0

In [75]:
resid['resid'] = preds - actuals

In [76]:
resid

,resid
2014-06-18,-0.028921
2014-06-19,0.440385
2014-06-20,-0.841351
2014-06-21,-0.744146
2014-06-22,-0.701409
2014-06-23,-0.286784
2014-06-24,1.253458
2014-06-25,0.837400
2014-06-26,0.786705
2014-06-27,0.375469


In [567]:
X_frame = macro[macro.index.isin(date_ranges[1])]

In [308]:
X = []

for vector in X_values.columns:
    X.append(np.array(X_values[vector]))

In [568]:
X = X_frame.values

In [569]:
len(X)

93

In [523]:
X = X.transpose()

In [526]:
X.shape

(28779, 93)

In [289]:
X_check = np.array(X)

In [291]:
X = X.transpose()

In [240]:
y = np.array(resid['resid'])

In [483]:
y

(92,)

In [484]:
X_inputs = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [485]:
X_inputs.shape

(92, 28779, 1)

In [456]:
input_shape = (28779,1)

In [477]:
gru_model.add(Input(shape=input_shape))

gru_model.add(GRU(units=100, return_sequences=True))
gru_model.add(Dropout(0.2))

gru_model.add(GRU(units=50, return_sequences=False))
gru_model.add(Dropout(0.2))
gru_model.add(Dense(1))
gru_model.add(Activation('linear'))

In [486]:
inputs = Input(shape=(28779,1))
gru_1 = GRU(units=100, return_sequences=True, dropout=0.2)(inputs)
gru_2 = GRU(units=50, return_sequences=False, dropout=0.2)(gru_1)
dense = Dense(1, activation='linear')(gru_2)

model = Model([inputs], dense)

In [487]:
model.compile(loss='mse', optimizer='rmsprop')

In [488]:
model.fit(x=X_inputs,y=y, batch_size=256, epochs = 3, validation_split=0.2)

Train on 73 samples, validate on 19 samples
Epoch 1/3
73/73 [==============================] - 118s 2s/step - loss: 30.7592 - val_loss: 37.1963
Epoch 2/3
73/73 [==============================] - 102s 1s/step - loss: 20.4083 - val_loss: 26.1914
Epoch 3/3
73/73 [==============================] - 92s 1s/step - loss: 18.3954 - val_loss: 21.8694


In [529]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 28779, 1)          0         
_________________________________________________________________
gru_103 (GRU)                (None, 28779, 100)        30600     
_________________________________________________________________
gru_104 (GRU)                (None, 50)                22650     
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 51        
Total params: 53,301
Trainable params: 53,301
Non-trainable params: 0
_________________________________________________________________


In [530]:
predictors_array = macro[macro.index.isin(date_ranges[2])].values.transpose()
predictors = np.reshape(predictors_array, (predictors_array.shape[1], predictors_array.shape[0], 1))

predictions = model.predict(predictors)

In [531]:
predictions.shape

(91, 1)

In [543]:
future = ts_model.make_future_dataframe(periods=len(date_ranges[1])+len(date_ranges[2]))
forecast = ts_model.predict(future)

ts_resid = forecast[forecast['ds'].isin(date_ranges[2])]['yhat'].values - etfs[etfs.index.isin(date_ranges[2])]['SPY'].values

In [563]:
predicted_prices = forecast[forecast['ds'].isin(date_ranges[2])]['yhat'].values

In [547]:
macro_predict = macro[macro.index.isin(date_ranges[2])].values
predictors = np.reshape(macro_predict, (macro_predict.shape[0], macro_predict.shape[1], 1))

predicted_resids = model.predict(predictors)

In [549]:
final_predictions = predicted_prices - predicted_resids

In [559]:
final_resids = predicted_resids.reshape(1,91)

In [564]:
trades = predicted_prices - final_resids[0]

In [566]:
etfs[etfs.index.isin(date_ranges[2])]['SPY'] - trades

date
2014-09-18    -3.828575
2014-09-19    -3.928543
2014-09-20    -4.025752
2014-09-21    -4.068493
2014-09-22    -5.845623
2014-09-23    -7.354370
2014-09-24    -6.322018
2014-09-25    -9.087831
2014-09-26    -7.596206
2014-09-27    -7.693424
2014-09-28    -7.736175
2014-09-29    -8.425415
2014-09-30    -9.367171
2014-10-01   -12.193229
2014-10-02   -11.987206
2014-10-03    -9.964846
2014-10-04   -10.061636
2014-10-05   -10.103963
2014-10-06   -10.673885
2014-10-07   -13.910130
2014-10-08   -11.204282
2014-10-09   -14.591504
2014-10-10   -16.537190
2014-10-11   -16.634024
2014-10-12   -16.676396
2014-10-13   -19.897960
2014-10-14   -20.098748
2014-10-15   -21.644341
2014-10-16   -21.612105
2014-10-17   -19.534931
                ...    
2014-11-18    -9.311606
2014-11-19    -9.998216
2014-11-20    -9.490894
2014-11-21    -8.419933
2014-11-22    -8.517218
2014-11-23    -8.560040
2014-11-24    -8.389854
2014-11-25    -8.993488
2014-11-26    -8.893826
2014-11-27    -8.715732
2014-11-28 

In [500]:
predicted_prices = actuals - predictions

In [501]:
predicted_prices

array([[173.30342632, 172.65592632, 173.87232632, ..., 176.63972632,
        178.00392632, 178.24962632],
       [173.30343013, 172.65593013, 173.87233013, ..., 176.63973013,
        178.00393013, 178.24963013],
       [173.30343443, 172.65593443, 173.87233443, ..., 176.63973443,
        178.00393443, 178.24963443],
       ...,
       [173.27978473, 172.63228473, 173.84868473, ..., 176.61608473,
        177.98028473, 178.22598473],
       [173.27878385, 172.63128385, 173.84768385, ..., 176.61508385,
        177.97928385, 178.22498385],
       [173.27779155, 172.63029155, 173.84669155, ..., 176.61409155,
        177.97829155, 178.22399155]])

In [424]:
gru_model.compile(loss='mse', optimizer='rmsprop')

In [425]:
model.fit(x=X,y=y, batch_size=256, epochs = 3, validation_split=0.2)

ValueError: Error when checking input: expected gru_84_input to have 3 dimensions, but got array with shape (28779, 92)